In [5]:
import pygame
import sys, os, getopt, math, random
import numpy as np
from constants import *

pygame 2.1.2 (SDL 2.0.18, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
swarm_size = 7  # default size of the swarm
manual_mode = False  # manually press enter key to proceed between simulations

In [7]:
def S14_closest_robot(robot_host, robot_neighbors):
    # "robot_host": the robot to measure distance from
    # "robot_neighbors": a list of robots to be compared with
    robot_closest = robot_neighbors[0]
    dist_closest = dist_table[robot_host,robot_closest]
    for i in robot_neighbors[1:]:
        dist_temp = dist_table[robot_host,i]
        if dist_temp < dist_closest:
            robot_closest = i
            dist_closest = dist_temp
    return robot_closest

In [574]:
def check_amoeba(poses_t, poses, num_robots, step, thresh = 1):
    centroid = getcenteroid(poses[:num_robots])
    centroid_t = getcenteroid(poses_t[:num_robots])
    print(centroid)
    arr_radii = np.array([get_dist(centroid, p) for p in poses[:num_robots]])
    arr_radii_t = np.array([get_dist(centroid, p) for p in poses_t[:num_robots]])
    
    delta_centroid = get_dist(centroid, centroid_t)
    delta_radius = np.mean(arr_radii) - np.mean(arr_radii_t)
    
    if delta_centroid/step < thresh and delta_radius/np.mean(arr_radii_t) < thresh:
        return True
    return False
    
    

# Function to get centroid
def getcenteroid(arr):
    length, dim = arr.shape
    return np.array([np.sum(arr[:, i])/length for i in range(dim)])

def get_dist(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

In [516]:
# general function to steer robot away from wall if out of boundary (following physics)
# use global variable "world_side_length"
def robot_boundary_check(robot_pos, robot_ori):
    new_ori = robot_ori
    if robot_pos[0] >= world_side_length:  # outside of right boundary
        if math.cos(new_ori) > 0:
            new_ori = reset_radian(2*(math.pi/2) - new_ori)
            # further check if new angle is too much perpendicular
            if new_ori > 0:
                if (math.pi - new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
            else:
                if (new_ori + math.pi) < perp_thres:
                    new_ori = new_ori + devia_angle
    elif robot_pos[0] <= 0:  # outside of left boundary
        if math.cos(new_ori) < 0:
            new_ori = reset_radian(2*(math.pi/2) - new_ori)
            if new_ori > 0:
                if new_ori < perp_thres:
                    new_ori = new_ori + devia_angle
            else:
                if (-new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
    if robot_pos[1] >= world_side_length:  # outside of top boundary
        if math.sin(new_ori) > 0:
            new_ori = reset_radian(2*(0) - new_ori)
            if new_ori > -math.pi/2:
                if (new_ori + math.pi/2) < perp_thres:
                    new_ori = new_ori + devia_angle
            else:
                if (-math.pi/2 - new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
    elif robot_pos[1] <= 0:  # outside of bottom boundary
        if math.sin(new_ori) < 0:
            new_ori = reset_radian(2*(0) - new_ori)
            if new_ori > math.pi/2:
                if (new_ori - math.pi/2) < perp_thres:
                    new_ori = new_ori + devia_angle
            else:
                if (math.pi/2 - new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
    return new_ori

# general function to reset radian angle to [-pi, pi)
def reset_radian(radian):
    while radian >= math.pi:
        radian = radian - 2*math.pi
    while radian < -math.pi:
        radian = radian + 2*math.pi
    return radian

In [10]:
# calculate world_side_coef from a desired screen size for 30 robots
def cal_world_side_coef():
    desired_screen_size = 400  # desired screen size for 30 robots
    desired_world_size = float(desired_screen_size) / pixels_per_length
    return desired_world_size / pow(3, 1/power_exponent)
world_side_coef = cal_world_side_coef()
world_side_length = world_side_coef * pow(swarm_size, 1/power_exponent)
world_size = (world_side_length, world_side_length)  # square physical world
# screen size calculated from world size
screen_side_length = int(pixels_per_length * world_side_length)
screen_size = (screen_side_length, screen_side_length)  # square display world

In [11]:
# Function to create a unit vector from a given vector
def unit_vector(vector):
    return np.abs(vector / np.linalg.norm(vector))

In [12]:
try:
    pygame.display.quit()
    pygame.quit()
except:
    pass
pygame.init()

font = pygame.font.SysFont("Cabin", 12)

screen = pygame.display.set_mode(screen_size)
pygame.display.set_caption("Demo 1")
# draw the network
screen.fill(color_white)

pygame.display.update()


In [441]:
def convert_coordinates(pos):
    poses_temp = pos / world_side_length
    poses_temp[1] = 1.0 - poses_temp[1]
    poses_temp = poses_temp * screen_side_length
    disp = poses_temp.astype(int) 
    return disp

In [560]:
screen.fill(color_white)


# # consensus configuration
# loop_folder = "loop-data2"  # folder to store the loop shapes
# shape_catalog = ["airplane", "circle", "cross", "goblet", "hand", "K", "lamp", "square",
#     "star", "triangle", "wrench"]
# shape_quantity = len(shape_catalog)  # the number of decisions
# shape_decision = -1  # the index of chosen decision, in range(shape_quantity)
#     # also the index in shape_catalog
# assignment_scheme = np.zeros(swarm_size)
# # variable to force shape to different choices, for video recording
# force_shape_set = range(shape_quantity)

step = 0.05

# robot properties
# robot_poses = np.random.rand(swarm_size, 2) * world_side_length  # initialize the robot poses
robot_poses = np.array([[1,1] for i in range(swarm_size)])  # initialize the robot poses
dist_table = np.zeros((swarm_size, swarm_size))  # distances between robots
conn_table = np.zeros((swarm_size, swarm_size))  # connections between robots
    # 0 for disconnected, 1 for connected
conn_lists = [[] for i in range(swarm_size)]  # lists of robots connected
# function for all simulations, update the distances and connections between the robots
def dist_conn_update():
    global dist_table
    global conn_table
    global conn_lists
    conn_lists = [[] for i in range(swarm_size)]  # empty the lists
    for i in range(swarm_size):
        for j in range(i+1, swarm_size):
            dist_temp = np.linalg.norm(robot_poses[i] - robot_poses[j])
            dist_table[i,j] = dist_temp
            dist_table[j,i] = dist_temp
            if dist_temp > comm_range:
                conn_table[i,j] = 0
                conn_table[j,i] = 0
            else:
                conn_table[i,j] = 1
                conn_table[j,i] = 1
                conn_lists[i].append(j)
                conn_lists[j].append(i)
dist_conn_update()  # update the distances and connections
disp_poses = []  # display positions
# function for all simulations, update the display positions
def disp_poses_update():
    global disp_poses
    poses_temp = robot_poses / world_side_length
    poses_temp[:,1] = 1.0 - poses_temp[:,1]
    poses_temp = poses_temp * screen_side_length
    disp_poses = poses_temp.astype(int)  # convert to int and assign to disp_poses
    # disp_poses = np.array([[0,0] for i in range(swarm_size)])  # 
disp_poses_update()
# deciding the seed robots, used in simulations with moving robots
seed_percentage = 0.1  # the percentage of seed robots in the swarm
seed_quantity = min(max(int(swarm_size*seed_percentage), 1), swarm_size)
    # no smaller than 1, and no larger than swarm_size
robot_seeds = [False for i in range(swarm_size)]  # whether a robot is a seed robot
    # only seed robot can initialize the forming a new group
seed_list_temp = np.arange(swarm_size)
np.random.shuffle(seed_list_temp)
for i in seed_list_temp[:seed_quantity]:
    robot_seeds[i] = True
    
current_swarm_size = 0  # the current swarm size
sim_haulted = False
# set up the simulation window

disp_poses_update()

robot_oris = np.random.rand(swarm_size) * 2 * math.pi - math.pi  # in range of [-pi, pi)


for i in range(swarm_size):
    pygame.draw.circle(screen, color_black, disp_poses[i], robot_size_formation,
        robot_width_empty)
    pygame.draw.circle(screen, color_black, disp_poses[i],
        int(comm_range*pixels_per_length), 1)
pygame.display.update()

robot_poses, current_swarm_size

(array([[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]]),
 0)

In [602]:
step = 2
for i in range(1):
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:  # close window button is clicked
            print("program exit in simulation 1")
            sys.exit()  # exit the entire program
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_SPACE:
                sim_haulted = not sim_haulted  # reverse the pause flag
    # if sim_haulted: continue
    print(f"Halted = {sim_haulted}, swarm size = {current_swarm_size}")
    print(f"current swarm size is: {current_swarm_size} out of {swarm_size}")
    current_pos = robot_poses[0]
    
    robot_poses_t = robot_poses.copy()
    if current_swarm_size == 0: pass #continue
    elif current_swarm_size == 1:
        # random_vector = unit_vector((np.random.rand(2)-0.5)*2)
        point = np.random.randint(10, size=2)
        robot_oris[i] = np.absolute(math.atan2(point[0], point[1]))
        
        robot_oris[i] = robot_boundary_check(robot_poses_t[i], robot_oris[i])
        robot_poses[i] = robot_poses_t[i] + (step * np.array([math.cos(robot_oris[i]), math.sin(robot_oris[i])]))
    else:
        direction = (np.sum(robot_poses[:, 0]), np.sum(robot_poses[:, 1]))
        
        for i in range(current_swarm_size): 
            while True:
                point = np.random.randint(-10, 10, size=2)
                robot_oris[i] = math.atan2(point[0], point[1])
                new = robot_poses.copy()
                robot_oris[i] = robot_boundary_check(robot_poses_t[i], robot_oris[i])
                new[i] = robot_poses_t[i] + (step * np.array([math.cos(robot_oris[i]), math.sin(robot_oris[i])]))
                if check_amoeba(robot_poses_t, new, current_swarm_size+1, step, thresh=0.9):
                    robot_poses = new
                    break
                
        
            
    print("robot positions: ", robot_poses) 
    
    if current_swarm_size < swarm_size:
        current_swarm_size+=1 
    disp_poses_update()
    screen.fill(color_white)
    
    for i in range(swarm_size):
        pygame.draw.circle(screen, color_black, disp_poses[i], robot_size_formation,
            robot_width_empty)
        pygame.draw.line(screen, color_grey, disp_poses[i], convert_coordinates(robot_poses[i]+np.array([math.cos(robot_oris[i]), math.sin(robot_oris[i])])*1))
        
        pygame.draw.circle(screen, color_black, disp_poses[i],
            int(comm_range*pixels_per_length), 1)
        
    # Plot centroid
    pygame.draw.circle(screen, color_red, convert_coordinates(getcenteroid(robot_poses[:current_swarm_size])), 4)
    pygame.display.update()

Halted = False, swarm size = 7
current swarm size is: 7 out of 7
[2.14285714 2.        ]
[1.85714286 2.        ]
[1.71428571 1.71428571]
[1.71428571 1.85714286]
[1.57142857 1.71428571]
[1.57142857 1.57142857]
[1.28571429 1.57142857]
robot positions:  [[3 6]
 [0 1]
 [0 0]
 [0 2]
 [0 1]
 [6 0]
 [0 1]]


In [119]:
dist_table

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [56]:
x

array([ 0.81159594, -0.58421916])

In [57]:
np.array([1,1])+x

array([1.81159594, 0.41578084])

In [581]:
disp_poses[0] + np.array([1, 1])

array([  1, 768])

In [63]:
robot_poses

array([[1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

In [64]:
random_vector = unit_vector((np.random.rand(2)-0.5)*2)
robot_poses[0] = current_pos + random_vector

In [148]:
math.atan2(-1, -1)

-2.356194490192345

In [577]:
robot_poses[:]

array([[0, 0],
       [2, 2],
       [0, 2],
       [1, 1],
       [0, 3],
       [0, 0],
       [1, 0]])